# Tryout *google* facets data visualization

- [github source code](https://github.com/pair-code/facets)
- [Installation guide to Jupyter](https://github.com/pair-code/facets#enabling-usage-in-jupyter-notebooks)

In [32]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [3]:
facets_path = os.path.dirname(os.path.join('..', '..', '..', '..'))
facets_path = os.path.abspath(os.path.join(facets_path, 'facets', 'facets_overview', 'python'))
                              
if not facets_path in sys.path:
    sys.path.append(facets_path)

In [4]:
from data_utils import METADATA_CSV, TRAIN_MASKS_CSV, OUTPUT_PATH, GENERATED_DATA

In [5]:
METADATA_CSV.head()

,id,year,make,model,trim1,trim2
0,0004d4463b50,2014.0,Acura,TL,TL,w/SE
1,00087a6bd4dc,2014.0,Acura,RLX,RLX,w/Tech
2,000aa097d423,2012.0,Mazda,MAZDA6,MAZDA6,i Sport
3,000f19f6e7d4,2016.0,Chevrolet,Camaro,Camaro,SS
4,00144e887ae9,2015.0,Acura,TLX,TLX,SH-AWD V6 w/Advance Pkg


## Metadata overview 

In [ ]:
from generic_feature_statistics_generator import GenericFeatureStatisticsGenerator
proto = GenericFeatureStatisticsGenerator().ProtoFromDataFrames([{'name': 'Metadata', 'table': METADATA_CSV}])

In [ ]:
from IPython.core.display import display, HTML
import base64
protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")
HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-jupyter.html" >
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display(HTML(html))

## Facets dive

Create sprite image (see [this](https://github.com/PAIR-code/facets/tree/master/facets_dive#providing-sprites-for-dive-to-render))

In [6]:
from data_utils import train_ids
from image_utils import get_image_data
import cv2
import numpy as np

In [25]:
atlas_image_path = os.path.abspath(os.path.join(GENERATED_DATA, "complete_atlas_image.jpg"))

_metadata_csv = METADATA_CSV#[METADATA_CSV['make'] == 'Acura']

sprit_image_size = (128, 128)
if not os.path.exists(atlas_image_path):
    ids = _metadata_csv['id'].values
    
    n = 100
    m = int(np.ceil(len(ids) * 1.0 / n))
    complete_image = np.zeros((m*sprit_image_size[0], n*sprit_image_size[1], 3), dtype=np.uint8)
    counter = 0
    for i in range(m):
        print("-- %i / %i" % (counter, len(ids)))
        ys = i*sprit_image_size[1]
        ye = ys + sprit_image_size[1]
        for j in range(n):
            xs = j*sprit_image_size[0]
            xe = xs + sprit_image_size[0]
            if counter == len(ids):
                break
            image_id = ids[counter] + '_03'; counter+=1
            image_type = 'Train' if image_id in train_ids else 'Test'
            img = get_image_data(image_id, image_type)
            img = cv2.resize(img, dsize=sprit_image_size)            
            complete_image[ys:ye, xs:xe, :] = img[:,:,:]
        if counter == len(ids):
            break        
    
    cv2.imwrite(atlas_image_path, complete_image)
    del complete_image
#     atlas_image

-- 0 / 6572
-- 100 / 6572
-- 200 / 6572
-- 300 / 6572
-- 400 / 6572
-- 500 / 6572
-- 600 / 6572
-- 700 / 6572
-- 800 / 6572
-- 900 / 6572
-- 1000 / 6572
-- 1100 / 6572
-- 1200 / 6572
-- 1300 / 6572
-- 1400 / 6572
-- 1500 / 6572
-- 1600 / 6572
-- 1700 / 6572
-- 1800 / 6572
-- 1900 / 6572
-- 2000 / 6572
-- 2100 / 6572
-- 2200 / 6572
-- 2300 / 6572
-- 2400 / 6572
-- 2500 / 6572
-- 2600 / 6572
-- 2700 / 6572
-- 2800 / 6572
-- 2900 / 6572
-- 3000 / 6572
-- 3100 / 6572
-- 3200 / 6572
-- 3300 / 6572
-- 3400 / 6572
-- 3500 / 6572
-- 3600 / 6572
-- 3700 / 6572
-- 3800 / 6572
-- 3900 / 6572
-- 4000 / 6572
-- 4100 / 6572
-- 4200 / 6572
-- 4300 / 6572
-- 4400 / 6572
-- 4500 / 6572
-- 4600 / 6572
-- 4700 / 6572
-- 4800 / 6572
-- 4900 / 6572
-- 5000 / 6572
-- 5100 / 6572
-- 5200 / 6572
-- 5300 / 6572
-- 5400 / 6572
-- 5500 / 6572
-- 5600 / 6572
-- 5700 / 6572
-- 5800 / 6572
-- 5900 / 6572
-- 6000 / 6572
-- 6100 / 6572
-- 6200 / 6572
-- 6300 / 6572
-- 6400 / 6572
-- 6500 / 6572


In [42]:
atlas_url = "/files/abc.jpg"

In [27]:
_metadata_csv.head()

,id,year,make,model,trim1,trim2
0,0004d4463b50,2014.0,Acura,TL,TL,w/SE
1,00087a6bd4dc,2014.0,Acura,RLX,RLX,w/Tech
2,000aa097d423,2012.0,Mazda,MAZDA6,MAZDA6,i Sport
3,000f19f6e7d4,2016.0,Chevrolet,Camaro,Camaro,SS
4,00144e887ae9,2015.0,Acura,TLX,TLX,SH-AWD V6 w/Advance Pkg


In [47]:
atlas_url = "../output/generated/complete_atlas_image.jpg"

In [48]:
# Display the Dive visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-jupyter.html">
        <facets-dive 
            id="elem" 
            height="750"
            cross-origin="anonymous"
            sprite-image-width="128"
            sprite-image-height="128">
        </facets-dive>
        <script>
          var data = {jsonstr};
          var atlas_url = "{atlas_url}";
          document.querySelector("#elem").data = data;
          document.querySelector("#elem").atlasUrl = atlas_url;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=_metadata_csv.to_json(orient='records'), atlas_url=atlas_url)
display(HTML(html))